<a href="https://colab.research.google.com/github/Anusree1221/Insurance/blob/main/Insurance_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load uploaded files
customers_df = pd.read_csv('/content/customers.csv')
products_df = pd.read_csv('/content/products.csv')

# Show data preview
print("Customers Data:")
display(customers_df)

print("\nProducts Data:")
display(products_df)


Customers Data:


,customer_id,age,marital_status,children,income,existing_products,smoker
0,1,32,married,2,70000,health,no
1,2,45,single,0,90000,life,yes
2,3,60,married,0,40000,NaN,no



Products Data:


,product_id,name,type,min_age,max_age,for_smokers,life_stage
0,101,Term Life Insurance,life,25,60,yes,family
1,102,Health Insurance,health,18,65,yes,all
2,103,Child Education Plan,education,30,50,no,family
3,104,Retirement Plan,retirement,50,70,yes,senior


In [2]:
def profile_customer(row):
    if row["age"] < 35 and row["marital_status"] == "single":
        return "young"
    elif row["marital_status"] == "married" and row["children"] > 0:
        return "family"
    elif row["age"] > 55:
        return "senior"
    else:
        return "adult"


In [3]:
def match_products(customer_row, products_df):
    life_stage = profile_customer(customer_row)
    age = customer_row["age"]
    smoker = customer_row["smoker"]

    matched = products_df[
        (products_df["min_age"] <= age) &
        (products_df["max_age"] >= age) &
        ((products_df["for_smokers"] == "yes") | (smoker == "no")) &
        ((products_df["life_stage"] == life_stage) | (products_df["life_stage"] == "all"))
    ].copy()

    matched["explanation"] = matched.apply(
        lambda p: f"Recommended because it fits your age ({age}) and life stage ({life_stage})",
        axis=1
    )
    return matched


In [4]:
def recommend(customer_id, customers_df, products_df):
    customer_row = customers_df[customers_df["customer_id"] == customer_id]

    if customer_row.empty:
        return pd.DataFrame(), "Customer not found."

    customer = customer_row.iloc[0]
    matched_products = match_products(customer, products_df)

    if matched_products.empty:
        return pd.DataFrame(), "No suitable products found."

    return matched_products[["product_id", "name", "type", "explanation"]], None


In [5]:
# Example usage
customer_id_to_test = 1
recommendations, message = recommend(customer_id_to_test, customers_df, products_df)

if message:
    print(message)
else:
    print(f"Recommendations for customer ID {customer_id_to_test}:")
    display(recommendations)


Recommendations for customer ID 1:


,product_id,name,type,explanation
0,101,Term Life Insurance,life,Recommended because it fits your age (32) and ...
1,102,Health Insurance,health,Recommended because it fits your age (32) and ...
2,103,Child Education Plan,education,Recommended because it fits your age (32) and ...


In [6]:
import pandas as pd

# Load customer and product data
customers = pd.read_csv('/content/customers.csv')
products = pd.read_csv('/content/customers.csv')

print("✔ Customers:")
display(customers.head())

print("\n✔ Products:")
display(products.head())


✔ Customers:


,customer_id,age,marital_status,children,income,existing_products,smoker
0,1,32,married,2,70000,health,no
1,2,45,single,0,90000,life,yes
2,3,60,married,0,40000,NaN,no



✔ Products:


,customer_id,age,marital_status,children,income,existing_products,smoker
0,1,32,married,2,70000,health,no
1,2,45,single,0,90000,life,yes
2,3,60,married,0,40000,NaN,no


In [7]:
def profile_customer(row):
    """
    Assign life stage based on age, marital status, and children.
    """
    if row["age"] < 35 and row["marital_status"] == "single":
        return "young"
    elif row["marital_status"] == "married" and row["children"] > 0:
        return "family"
    elif row["age"] > 55:
        return "senior"
    else:
        return "adult"


In [8]:
def assess_needs(customer):
    """
    Identify broad insurance needs based on life stage.
    """
    life_stage = profile_customer(customer)
    needs = []

    if life_stage == "young":
        needs.extend(["health", "term_life"])
    elif life_stage == "family":
        needs.extend(["health", "term_life", "child_plan", "home"])
    elif life_stage == "senior":
        needs.extend(["health", "retirement", "critical_illness"])
    else:
        needs.extend(["health", "life"])

    return needs


In [9]:
def match_products(customer, products):
    """
    Filters and matches insurance products to customer profile.
    """
    age = customer["age"]
    smoker = customer["smoker"]
    life_stage = profile_customer(customer)
    needs = assess_needs(customer)

    matched = products[
        (products["min_age"] <= age) &
        (products["max_age"] >= age) &
        ((products["for_smokers"] == "yes") | (smoker == "no")) &
        ((products["life_stage"] == life_stage) | (products["life_stage"] == "all")) &
        (products["type"].isin(needs))
    ].copy()

    matched["explanation"] = matched.apply(
        lambda p: f"Matched for age {age}, life stage '{life_stage}', and need '{p['type']}'", axis=1
    )

    return matched


def recommend_insurance(customer_id):
    """
    Recommend insurance products with explanations for a given customer.
    """
    row = customers[customers["customer_id"] == customer_id]
    if row.empty:
        return f"❌ No customer found with ID {customer_id}", pd.DataFrame()

    customer = row.iloc[0]
    recommendations = match_products(customer, products)

    if recommendations.empty:
        return f"⚠️ No suitable products found for customer ID {customer_id}", pd.DataFrame()

    return f"✅ Recommendations for customer ID {customer_id}", recommendations[["product_id", "name", "type", "explanation"]]


In [13]:
try:
    # Set the customer ID to test
    customer_id_to_test = 1

    # Run the recommendation function
    message, recommendations = recommend_insurance(customer_id_to_test)

    # Output results
    print(message)
    if not recommendations.empty:
        display(recommendations)
    else:
        print("No recommendations found.")
except Exception as e:
    print("❌ Error occurred while running recommendation engine:")
    print(str(e))


❌ Error occurred while running recommendation engine:
'min_age'


In [14]:
# Test recommendation for customer ID 1
message, recommended_products = recommend_insurance(1)
print(message)
display(recommended_products)


KeyError: 'min_age'

In [15]:
# Debug customer
print("Sample customer row:")
print(customers[customers["customer_id"] == 1])

# Debug profile
print("Life stage profile:", profile_customer(customers[customers["customer_id"] == 1].iloc[0]))


Sample customer row:
   customer_id  age marital_status  children  income existing_products smoker
0            1   32        married         2   70000            health     no
Life stage profile: family
